### Project Chicago: Transform ResStock PACKAGE Results to 15 SFD prototypes (Elevate Energy)
Created on: 01/07/2020 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/ResStock/files


### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/chicagoeui
* processed result tables for plots: https://nrel.sharepoint.com/sites/ChicagoRetrofits/Shared%20Documents/Forms/AllItems.aspx?viewid=289cdd1a%2D97c9%2D4bcc%2D8416%2Dc19bf01c6302&id=%2Fsites%2FChicagoRetrofits%2FShared%20Documents%2FGeneral%2FUpgrade%20results

### Initialize
For modifying plots, can go directly to section 2.3 after initialization

In [2]:
# local path to downloaded results
iteration = '02_sfamh' # <----- options: 02_sfamh
iter_path = f'cookcnty_packages_{iteration}'
result_dir = '/Users/lliu2/Documents/Chicago retrofits/ResStock results'
result_path = os.path.join(result_dir, iter_path)

# accessory file path (for adding to package iteration)
acc_iter_path = f'cookcnty_upgrades_02'
acc_path = os.path.join(result_dir, acc_iter_path)

## create folder for post-processed results:
if not os.path.exists(os.path.join(result_path, 'processed results')):
    os.mkdir(os.path.join(result_path, 'processed results'))
    
## create folder for exported baseline and upgrade results to csv (to share with Elevate):
combined_res_csv_path = os.path.join(result_path, 'processed results', 'raw combined csvs')
if not os.path.exists(combined_res_csv_path):
    os.mkdir(combined_res_csv_path)
    
## create folder for plots:
plot_path = os.path.join(result_path, 'processed results', 'plots')
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

print(f'Results path: \n   {result_path}')
print(f'Accessory files path: \n   {acc_path}')


Results path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh
Accessory files path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_upgrades_02


### Functions

In [3]:
def get_per_unit_sim_output(df, ref):
    """
    ref (df): baseline df
    """
    cols = [x for x in df.columns if 
            x.endswith('_kwh') or
            x.endswith('_mbtu') or
            x.endswith('_therm') or
            x.endswith('_cost_usd') or
            x.endswith('_ft_2')
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output"')

def get_per_unit_sim_output_limited(df, ref):
    """
    To reduce computing time
    ref (df): baseline df
    """
    cols = ['simulation_output_report.total_site_natural_gas_therm',
            'simulation_output_report.total_site_electricity_kwh',
            'simulation_output_report.total_site_energy_mbtu',
            'simulation_output_report.upgrade_cost_usd'
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output_limited"')

def add_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    df['gas_eui_thermpersqft'] = df['simulation_output_report.total_site_natural_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['simulation_output_report.total_site_electricity_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_kbtupersqft'] = df['simulation_output_report.total_site_energy_mbtu'].divide(df['sqft'])*1000 # kbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_sqft_eui"')

def get_res_by_prototype(df, filter_by_df, row):
    """
    ARG:
        df (dataframe): df to slice on
        filter_by_df (dataframe): df used to do the slicing
    RETURN:
        filtered df (dataframe)
    """
    slice_by_df = filter_by_df.copy().set_index('building_id').reindex(df['building_id']).reset_index()
    res_group_i = df[slice_by_df['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        slice_by_df['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        slice_by_df['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        slice_by_df['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    res_group_i = res_group_i[res_group_i['completed_status']=="Success"]
    
    return res_group_i
print('func loaded: "get_res_by_prototype"')

def load_upgrade(n, file_dir=result_path):
    """
    ARG:
        n (int, str): upgrade number
        file_dir (str): folder in which upgrade can be found, default to main result dir
    RETURN:
        df (dataframe) of upgrade n
    """
    df = pd.read_parquet(os.path.join(file_dir,'upgrades',
                                     f'upgrade={n}/results_up{n:02d}.parquet'))
    return df
print('func loaded: "load_upgrade"')



func loaded: "get_per_unit_sim_output"
func loaded: "get_per_unit_sim_output_limited"
func loaded: "add_sqft_eui"
func loaded: "get_res_by_prototype"
func loaded: "load_upgrade"


### 1. BASELINE results

In [4]:
save_a_copy_in_csv = False # <-----
res = pd.read_parquet(os.path.join(result_path,'baseline','results_up00.parquet'))

# (1) get sqft, gas/elec/site eui
res = add_sqft_eui(res, res)
res['build_existing_model.sample_weight'] = 2173432/40000

if save_a_copy_in_csv:
    res.to_csv(os.path.join(combined_res_csv_path,'results_baseline.csv'), index=False)
    
# (1) get sim output at the unit level (req for MF)
res = get_per_unit_sim_output(res, res)

# (2) get sqft, gas/elec/site eui - redo to get unit-level results
res = add_sqft_eui(res, res)

# check
jobs_missing = set(range(1,246)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {16777-len(res)} buildings ({((16777-len(res))/16777*100):.2f}%)')

res

- 0 jobs missing: set()
- 14313 buildings (85.31%)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft
0,1,74,2021-03-16 13:37:14,2021-03-16 13:41:12,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.909091,18.272727,336.746701,319.046765,2514.0,395.288816,1202.0,0.445364,4.098369,58.509988
1,2,36,2021-03-16 12:25:01,2021-03-16 12:28:13,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.090909,16.363636,53.244335,67.353566,2155.0,74.409253,866.0,0.875912,7.004846,111.471826
2,3,42,2021-03-16 13:03:52,2021-03-16 13:07:45,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.000000,17.363636,141.755328,90.612860,4744.0,170.926109,1202.0,0.447211,3.904397,58.032766
3,4,107,2021-03-16 14:01:48,2021-03-16 14:05:09,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.454545,14.363636,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885
4,5,79,2021-03-16 14:00:54,2021-03-16 14:04:54,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,13.272727,13.909091,257.154109,278.262128,6378.0,376.545386,1675.0,0.288642,5.129431,46.359652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,217,2021-03-16 14:23:12,2021-03-16 14:24:31,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.727273,18.272727,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562
2460,2461,213,2021-03-16 14:11:20,2021-03-16 14:15:11,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.727273,18.818182,114.455744,84.153168,3955.0,132.511084,1675.0,0.515953,3.816256,64.604648
2461,2462,214,2021-03-16 13:28:31,2021-03-16 13:32:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.181818,16.363636,47.367171,25.410279,4456.0,52.041645,1202.0,0.876150,4.199488,101.923318
2462,2463,94,2021-03-16 13:04:33,2021-03-16 13:08:38,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.727273,17.272727,103.762542,79.446300,5005.0,117.707768,1675.0,0.678681,4.002326,81.508456


In [5]:
# assign heating/cooling
cooling_col = 'build_existing_model.hvac_cooling_type'
heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')

# if using a national run
if iteration == '_national_2018':
    res = res[res['build_existing_model.ahs_region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res
    

selected


### 1.1. Check housing charateristics distributions in BASELINE

In [6]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iter_path} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, 'baseline', 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, "baseline")}')

>>> ResStock - cookcnty_packages_02_sfamh - BASELINE result summary:

  * 2464 / 2464 samples ran successfully, 0 failed, efficacy: 100.0% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1           0.489854
2           0.455763
3           0.054383
N_failed    0.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry     0.505682
WoodStud    0.494318
N_failed    0.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59     0.136769
1960-79     0.242289
1980-99     0.353896
2000-09     0.164367
2010s       0.014610
<1940       0.088068
N_failed    0.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC    0.807630
Heat Pump     0.025162
None          0.026380
Room AC       0.140828
N_failed      0.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP                   

### 1.2. Add additional metrics to Baseline

In [7]:
## EE prototype tags
res['vintage_ee'] = '3: post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '2: 1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = '1: pre-1942'

res['stories'] = '2: 2+ stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = '1: <2 stories'

print('new cols added to "res"')
res

new cols added to "res"


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,74,2021-03-16 13:37:14,2021-03-16 13:41:12,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,336.746701,319.046765,2514.0,395.288816,1202.0,0.445364,4.098369,58.509988,3: post-1978,2: 2+ stories
1,2,36,2021-03-16 12:25:01,2021-03-16 12:28:13,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,53.244335,67.353566,2155.0,74.409253,866.0,0.875912,7.004846,111.471826,2: 1942-1978,1: <2 stories
2,3,42,2021-03-16 13:03:52,2021-03-16 13:07:45,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,141.755328,90.612860,4744.0,170.926109,1202.0,0.447211,3.904397,58.032766,3: post-1978,2: 2+ stories
3,4,107,2021-03-16 14:01:48,2021-03-16 14:05:09,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885,3: post-1978,1: <2 stories
4,5,79,2021-03-16 14:00:54,2021-03-16 14:04:54,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,257.154109,278.262128,6378.0,376.545386,1675.0,0.288642,5.129431,46.359652,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,217,2021-03-16 14:23:12,2021-03-16 14:24:31,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562,1: pre-1942,1: <2 stories
2460,2461,213,2021-03-16 14:11:20,2021-03-16 14:15:11,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,114.455744,84.153168,3955.0,132.511084,1675.0,0.515953,3.816256,64.604648,2: 1942-1978,2: 2+ stories
2461,2462,214,2021-03-16 13:28:31,2021-03-16 13:32:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,47.367171,25.410279,4456.0,52.041645,1202.0,0.876150,4.199488,101.923318,3: post-1978,2: 2+ stories
2462,2463,94,2021-03-16 13:04:33,2021-03-16 13:08:38,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,103.762542,79.446300,5005.0,117.707768,1675.0,0.678681,4.002326,81.508456,3: post-1978,2: 2+ stories


### 1.3. Get BASELINE results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags directly to ResStock results as prototypes overlap in chars

In [8]:
## Load prototype csv
groups = pd.read_csv(os.path.join(result_dir,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [9]:
### (1) get count & median consumption
Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.loc[i, 'count'] = len(res_group_i)
        else:
            groups.loc[i, metric] = res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> (1) median consumption computed')


>> (1) median consumption computed


In [10]:
res2 = []

for i, row in groups.iterrows():
    if row['HousingGroupName'] not in \
        ['Masonry All Years Split Level','Frame Post-1978 Split Level','Frame Pre-1942 Split Level']:
        res_group_i = get_res_by_prototype(res, res, row) # <----
        res_group_i['prototype'] = row['HousingGroupName']
        res2.append(res_group_i)

res2 = pd.concat(res2, axis=0).sort_values(by=['building_id'])
display(res2)

res2 = res2.set_index('building_id').reindex(res['building_id']).reset_index()
res.loc[res2['prototype'].isnull(),:]

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories,prototype


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,74,2021-03-16 13:37:14,2021-03-16 13:41:12,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,336.746701,319.046765,2514.0,395.288816,1202.0,0.445364,4.098369,58.509988,3: post-1978,2: 2+ stories
1,2,36,2021-03-16 12:25:01,2021-03-16 12:28:13,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,53.244335,67.353566,2155.0,74.409253,866.0,0.875912,7.004846,111.471826,2: 1942-1978,1: <2 stories
2,3,42,2021-03-16 13:03:52,2021-03-16 13:07:45,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,141.755328,90.612860,4744.0,170.926109,1202.0,0.447211,3.904397,58.032766,3: post-1978,2: 2+ stories
3,4,107,2021-03-16 14:01:48,2021-03-16 14:05:09,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885,3: post-1978,1: <2 stories
4,5,79,2021-03-16 14:00:54,2021-03-16 14:04:54,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,257.154109,278.262128,6378.0,376.545386,1675.0,0.288642,5.129431,46.359652,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,217,2021-03-16 14:23:12,2021-03-16 14:24:31,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562,1: pre-1942,1: <2 stories
2460,2461,213,2021-03-16 14:11:20,2021-03-16 14:15:11,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,114.455744,84.153168,3955.0,132.511084,1675.0,0.515953,3.816256,64.604648,2: 1942-1978,2: 2+ stories
2461,2462,214,2021-03-16 13:28:31,2021-03-16 13:32:24,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,47.367171,25.410279,4456.0,52.041645,1202.0,0.876150,4.199488,101.923318,3: post-1978,2: 2+ stories
2462,2463,94,2021-03-16 13:04:33,2021-03-16 13:08:38,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,103.762542,79.446300,5005.0,117.707768,1675.0,0.678681,4.002326,81.508456,3: post-1978,2: 2+ stories


In [11]:
### (2) get mean consumption
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    for metric, res_var in Metric_map2.items():
        groups.loc[i, metric] = res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> (2) mean consumption computed')


>> (2) mean consumption computed


In [12]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui_thermpersqft', 
               'median elec eui': 'elec_eui_kwhpersqft'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui_thermpersqft', 
               'mean elec eui': 'elec_eui_kwhpersqft'}

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.loc[i, metric] = res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.loc[i, metric] = \
            res_group_i[res_var].mean()

print('>> (3) median & mean sqft & eui computed:');

groups.set_index(['HousingGroupNo','HousingGroupName'])[['median sqft', 'mean sqft',
                                                         'median gas','mean gas',
                                                         'median gas eui', 'mean gas eui', 
                                                         'median elec','mean elec', 
                                                         'median elec eui','mean elec eui']]


>> (3) median & mean sqft & eui computed:


,,median sqft,mean sqft,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
HousingGroupNo,HousingGroupName,,,,,,,,,,
1,Masonry Post-1978 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Masonry All Years Split Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Masonry Post-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Masonry 1942-1978 Less than 2 Stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Masonry 1942-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Masonry Pre-1942 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Masonry Pre-1942 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Frame Post-1978 Less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Frame Post-1978 Split Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### export BASELINE prototype summary
groups.to_csv(os.path.join(result_path, 'baseline', 'baseline_prototype_results.csv'), index=True)
print(f'BASELINE prototype summary saved to:\n  {os.path.join(result_path, "baseline")}')


BASELINE prototype summary saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/baseline


## 2A. PACKAGE post processing ###
Three packages were ran as partitioned files. See "Cookcnty_packages_02.yml" for info
#### (1) Packages WITHOUT partitions (N=1, pkg 06)

In [14]:
def check_and_correct_for_missing_buildings(Pi, res, upgrade_no):
    delta = len(res)-len(Pi)
    if delta>0:
        print(f'   - upgrade_{upgrade_no:02d} is missing {delta} buildings compared to baseline')
    # always reindex
    Pi = Pi.set_index('building_id').reindex(res['building_id']).reset_index()
        
    return Pi

'func "check_and_correct_for_missing_buildings" loaded'

'func "check_and_correct_for_missing_buildings" loaded'

In [15]:
### (1) 
upgrade_list = [1,]
package_numbers = [6,]

print(f'packages without partitions: {len(upgrade_list)}')

for up, pk in zip(upgrade_list, package_numbers):
    file = f'package{pk:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}')

    if os.path.exists(filename):
        Pi = pd.read_csv(filename)

    else:
        Pi = load_upgrade(up, result_path)
        Pi = check_and_correct_for_missing_buildings(Pi, res, up)

        ## (3) save
        Pi.to_csv(filename, index=False)
    display(Pi)
    
print(f'\n>> file saved to: {combined_res_csv_path}')

packages without partitions: 1
* package06.csv


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,167,2021-03-16 14:00:39,2021-03-16 14:07:21,Success,True,Utility Incentive Shell + LEDs,NaN,True,600.0,...,131.063028,33.983223,34.316186,28.712445,16.909091,18.272727,310.617582,282.134740,2514.0,354.186739
1,2,17,2021-03-16 12:15:34,2021-03-16 12:20:15,Success,True,Utility Incentive Shell + LEDs,NaN,True,200.0,...,37.538012,19.553228,18.875889,19.530540,13.727273,14.181818,49.095663,61.096065,2155.0,65.623068
2,3,23,2021-03-16 13:10:52,2021-03-16 13:15:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,400.0,...,69.725440,33.539668,25.251718,24.192686,16.000000,17.181818,135.817499,86.565125,4744.0,167.639993
3,4,122,2021-03-16 13:18:18,2021-03-16 13:23:48,Success,True,Utility Incentive Shell + LEDs,NaN,True,120.0,...,70.120897,7.301510,35.699110,6.482665,13.363636,14.363636,247.421165,338.496207,544.0,420.639854
4,5,107,2021-03-16 13:48:18,2021-03-16 13:54:05,Success,True,Utility Incentive Shell + LEDs,NaN,True,160.0,...,95.976310,19.471876,16.077044,15.425625,13.272727,13.909091,253.268398,273.163713,6377.0,367.100372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,179,2021-03-16 13:06:44,2021-03-16 13:10:07,Success,True,Utility Incentive Shell + LEDs,NaN,True,20.0,...,4.807971,2.357085,2.353224,2.329461,16.727273,17.000000,5.684632,7.516141,8561.0,8.372389
2460,2461,10,2021-03-16 12:47:30,2021-03-16 12:53:32,Success,True,Utility Incentive Shell + LEDs,NaN,True,240.0,...,62.689861,28.852919,18.938970,19.426094,15.727273,18.818182,112.133549,82.067516,3955.0,128.473908
2461,2462,29,2021-03-16 12:51:40,2021-03-16 12:57:46,Success,True,Utility Incentive Shell + LEDs,NaN,True,100.0,...,14.990917,5.309824,6.698447,5.313598,14.545455,16.181818,45.366538,23.016742,4456.0,51.839938
2462,2463,201,2021-03-16 13:17:10,2021-03-16 13:22:55,Success,True,Utility Incentive Shell + LEDs,NaN,True,160.0,...,32.106046,15.390576,12.509278,11.160146,14.000000,12.363636,84.315861,60.167024,5005.0,97.353097



>> file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


#### (2) Packages WITH single-level partitions (N=10, pkg 07-16)

In [16]:
# 2 partitiions over 'Attic Insulation'

def combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                           MSHP_option=None, nullify_total_ng=False, save_as_csv=True, 
                                           recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        partition_para (str): para to filter the upgrades by and combine
        options_list_for_first_upgrades (list of list): list of para options to filter the first n-1 upgrades by
        MSHP_option (int): option number in package to correct, default to None
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N = len(upgrade_list)
    partition_para = f'build_existing_model.{partition_para}'
    
    # check setting:
    if N-1 != len(options_list_for_first_upgrades):
        raise ValueError(f'The size of "options_list_for_first_upgrades" = {len(options_list_for_first_upgrades)}, '+
                         f'is not 1 less than the size of "upgrade_list" = {N}')

    if not os.path.exists(filename) or recreate_from_scratch:
        print(f'building from resstock results... partitioned by:\n  "{partition_para}"')

        ## (1) building ids for each partitions based on partition_para and para_options
        bldg_list = options_list = []; msg = ' '; n_bldgs = 0
        for n in range(N-1):
            bldgs_for_part_n = res[res[partition_para].isin(options_list_for_first_upgrades[n])]['building_id']
            bldg_list.append(bldgs_for_part_n)
            
            options_for_part_n = options_list_for_first_upgrades[n]
            options_list = options_list + options_for_part_n
            
            n_bldgs += len(bldgs_for_part_n)
            msg += f' part {n+1}: {len(bldgs_for_part_n)},'
        
        # for last partition:
        bldgs_for_part_n = res[~res[partition_para].isin(options_list)]['building_id']
        bldg_list.append(bldgs_for_part_n)
        
        n_bldgs += len(bldgs_for_part_n)
        msg += f' part {N}: {len(bldgs_for_part_n)}, total: {n_bldgs}'
        print(msg)

        ## (2) combine partitions and update upgrade name
        P1 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P1.append(p[p['building_id'].isin(Bi)])

        P1 = pd.concat(P1, axis=0).sort_index()

        P1['apply_upgrade.upgrade_part'] = P1['apply_upgrade.upgrade_name'] # new col to show partition #
        P1['apply_upgrade.upgrade_name'] = P1['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P1.columns:
                P1['simulation_output_report.upgrade_cost_usd_orig'] = P1['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P1['simulation_output_report.upgrade_cost_usd'] = P1['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P1['simulation_output_report.upgrade_cost_usd'] -= P1[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P1['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P1.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P1.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P1['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P1.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P1['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, MSHP_cost]

            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P1.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P1.loc[P1['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (5) save
        if save_as_csv:
            print(f'\n>> combined file saved to: {combined_res_csv_path}')
            P1.to_csv(filename, index=False)
            
    else:
        P1 = pd.read_csv(filename)

#     display(P1)
    return P1

print('func "combine_df_from_two_partitions"')


func "combine_df_from_two_partitions"


In [17]:
pkg_no = 7
upgrade_list = [2, 3]
partition_para = 'insulation_unfinished_attic'
options_list_for_first_upgrades = [
    ['Uninsulated, Vented','Ceiling R-7, Vented','Ceiling R-13, Vented'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package07.csv, from upgrades: [2, 3]
building from resstock results... partitioned by:
  "build_existing_model.insulation_unfinished_attic"
  part 1: 440, part 2: 2024, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,151,2021-03-16 13:24:24,2021-03-16 13:31:01,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,600.0,...,34.065441,34.542305,28.713549,16.909091,18.272727,311.157309,282.385408,2514.0,354.272259,Utility Incentive Shell + LEDs - Forced Air - ...
1,2,81,2021-03-16 13:16:41,2021-03-16 13:17:25,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
2,3,173,2021-03-16 14:02:24,2021-03-16 14:08:35,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,400.0,...,33.543938,25.433279,24.194957,16.000000,17.272727,135.858225,85.874198,4744.0,167.733857,Utility Incentive Shell + LEDs - Forced Air - ...
3,4,199,2021-03-16 14:07:42,2021-03-16 14:08:25,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
4,5,101,2021-03-16 13:32:30,2021-03-16 13:39:00,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,160.0,...,19.453764,16.079690,15.418508,13.272727,13.909091,253.302139,273.162298,6377.0,367.233999,Utility Incentive Shell + LEDs - Forced Air - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,96,2021-03-16 13:12:38,2021-03-16 13:13:21,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
2460,2461,99,2021-03-16 13:30:33,2021-03-16 13:31:19,Invalid,True,Utility Incentive Shell + LEDs - Forced Air,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs - Forced Air - ...
2461,2462,187,2021-03-16 14:14:23,2021-03-16 14:21:40,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,100.0,...,5.309824,6.743550,5.313734,14.545455,16.272727,45.494340,23.084802,4456.0,51.862917,Utility Incentive Shell + LEDs - Forced Air - ...
2462,2463,213,2021-03-16 13:35:34,2021-03-16 13:41:53,Success,True,Utility Incentive Shell + LEDs - Forced Air,,True,160.0,...,15.385430,12.632419,11.158563,14.000000,12.363636,84.582801,60.379378,5005.0,97.689881,Utility Incentive Shell + LEDs - Forced Air - ...


In [18]:
pkg_no = 8
upgrade_list = [4, 5]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package08.csv, from upgrades: [4, 5]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,21,2021-03-16 13:07:57,2021-03-16 13:16:29,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,600.0,...,33.190513,33.698658,28.361912,16.909091,18.272727,303.653137,280.951255,2514.0,346.695207,Utility Incentive Shell + LEDs + Appliances - ...
1,2,177,2021-03-16 13:10:36,2021-03-16 13:11:22,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2,3,91,2021-03-16 14:02:30,2021-03-16 14:09:54,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,400.0,...,30.889310,24.916016,23.227095,16.000000,17.181818,129.150827,84.850598,4744.0,157.161079,Utility Incentive Shell + LEDs + Appliances - ...
3,4,72,2021-03-16 13:39:41,2021-03-16 13:40:22,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
4,5,212,2021-03-16 14:18:14,2021-03-16 14:25:53,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,160.0,...,18.968368,16.003540,15.214458,14.363636,13.909091,176.703173,186.217320,6377.0,246.952574,Utility Incentive Shell + LEDs + Appliances - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,170,2021-03-16 13:41:01,2021-03-16 13:41:46,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2460,2461,92,2021-03-16 12:54:50,2021-03-16 12:55:36,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2461,2462,75,2021-03-16 12:49:17,2021-03-16 12:57:29,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,100.0,...,5.137025,6.388709,5.149095,15.000000,16.727273,42.264713,21.308870,4456.0,48.215715,Utility Incentive Shell + LEDs + Appliances - ...
2462,2463,7,2021-03-16 12:19:49,2021-03-16 12:27:57,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,160.0,...,14.338249,11.979310,10.806291,13.454545,12.818182,78.531519,55.950479,5005.0,93.069126,Utility Incentive Shell + LEDs + Appliances - ...


In [19]:
pkg_no = 9
upgrade_list = [6, 7]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package09.csv, from upgrades: [6, 7]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,101,2021-03-16 14:00:47,2021-03-16 14:01:27,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
1,2,188,2021-03-16 13:13:14,2021-03-16 13:13:57,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
2,3,2,2021-03-16 12:35:46,2021-03-16 12:36:30,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
3,4,18,2021-03-16 13:08:18,2021-03-16 13:09:00,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 2
4,5,153,2021-03-16 14:03:41,2021-03-16 14:04:26,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,67,2021-03-16 13:48:25,2021-03-16 13:53:02,Success,True,Utility Incentive Shell + WH - Hydronic,,True,20.0,...,2.388173,2.398792,2.363308,16.727273,17.000000,5.715720,7.556628,8561.0,8.419336,Utility Incentive Shell + WH - Hydronic - Part 1
2460,2461,4,2021-03-16 12:16:39,2021-03-16 12:23:24,Success,True,Utility Incentive Shell + WH - Hydronic,,True,240.0,...,27.889827,19.324650,19.661183,15.818182,18.818182,110.868469,82.513156,3955.0,125.902696,Utility Incentive Shell + WH - Hydronic - Part 1
2461,2462,18,2021-03-16 12:37:47,2021-03-16 12:38:34,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1
2462,2463,233,2021-03-16 13:20:47,2021-03-16 13:21:29,Invalid,True,Utility Incentive Shell + WH - Hydronic,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + WH - Hydronic - Part 1


In [20]:
pkg_no = 10
upgrade_list = [8, 9]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package10.csv, from upgrades: [8, 9]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,218,2021-03-16 14:12:56,2021-03-16 14:13:41,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
1,2,8,2021-03-16 12:55:04,2021-03-16 12:55:50,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2,3,203,2021-03-16 13:30:42,2021-03-16 13:31:24,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
3,4,167,2021-03-16 13:22:27,2021-03-16 13:23:14,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
4,5,134,2021-03-16 12:55:33,2021-03-16 12:56:19,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,62,2021-03-16 13:18:13,2021-03-16 13:23:18,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,20.0,...,2.346829,2.363396,2.323028,16.727273,17.181818,5.382696,7.215101,8561.0,8.056213,Utility Incentive Shell + LEDs + Appliances - ...
2460,2461,115,2021-03-16 14:01:43,2021-03-16 14:09:47,Success,True,Utility Incentive Shell + LEDs + Appliances - ...,,True,240.0,...,24.973194,17.519165,17.513545,15.818182,18.818182,106.773730,79.702230,3955.0,122.230140,Utility Incentive Shell + LEDs + Appliances - ...
2461,2462,164,2021-03-16 12:59:34,2021-03-16 13:00:17,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...
2462,2463,231,2021-03-16 14:01:34,2021-03-16 14:02:16,Invalid,True,Utility Incentive Shell + LEDs + Appliances - ...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Utility Incentive Shell + LEDs + Appliances - ...


In [21]:
pkg_no = 11
upgrade_list = [10, 11]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package11.csv, from upgrades: [10, 11]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,197,2021-03-16 13:30:42,2021-03-16 13:38:16,Success,True,Comprehensive - Forced Air + WH + Shell,,True,600.0,...,35.544295,34.340814,29.692592,16.909091,18.272727,305.231758,281.803039,2514.0,348.516604,Comprehensive - Forced Air + WH + Shell - Part 1
1,2,100,2021-03-16 13:30:02,2021-03-16 13:30:46,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2,3,115,2021-03-16 12:54:38,2021-03-16 13:02:03,Success,True,Comprehensive - Forced Air + WH + Shell,,True,400.0,...,32.462857,25.408381,24.494951,16.000000,17.181818,129.827597,85.655256,4744.0,160.495734,Comprehensive - Forced Air + WH + Shell - Part 1
3,4,151,2021-03-16 14:06:15,2021-03-16 14:06:56,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 2
4,5,57,2021-03-16 13:35:31,2021-03-16 13:42:49,Success,True,Comprehensive - Forced Air + WH + Shell,,True,160.0,...,20.055632,15.963126,16.037738,13.272727,13.909091,252.150757,273.186684,6377.0,367.282650,Comprehensive - Forced Air + WH + Shell - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,189,2021-03-16 13:55:16,2021-03-16 13:56:00,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2460,2461,83,2021-03-16 13:54:06,2021-03-16 13:54:56,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2461,2462,89,2021-03-16 13:19:51,2021-03-16 13:27:16,Success,True,Comprehensive - Forced Air + WH + Shell,,True,100.0,...,5.441984,6.581535,5.443743,14.818182,16.181818,42.915175,22.437652,4456.0,50.976200,Comprehensive - Forced Air + WH + Shell - Part 1
2462,2463,231,2021-03-16 14:26:40,2021-03-16 14:34:07,Success,True,Comprehensive - Forced Air + WH + Shell,,True,160.0,...,15.258633,12.296444,11.418811,13.636364,12.363636,77.475789,59.813106,5005.0,91.576118,Comprehensive - Forced Air + WH + Shell - Part 1


In [22]:
pkg_no = 12
upgrade_list = [12, 13, 14]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50'],
    ['10 ACH50','8 ACH50','7 ACH50','6 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package12.csv, from upgrades: [12, 13, 14]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 1738, part 2: 599, part 3: 127, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,91,2021-03-16 13:39:32,2021-03-16 13:47:43,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,600.0,...,34.780907,34.749733,29.624955,15.181818,13.818182,394.697126,406.155502,4815.0,475.289046,Comprehensive - Forced Air + WH + Insulation +...
1,2,6,2021-03-16 12:44:16,2021-03-16 12:44:59,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
2,3,131,2021-03-16 12:46:31,2021-03-16 12:54:57,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,400.0,...,32.126188,25.608729,24.202425,16.454545,18.454545,178.407022,148.172914,4744.0,227.208179,Comprehensive - Forced Air + WH + Insulation +...
3,4,115,2021-03-16 13:59:00,2021-03-16 13:59:42,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
4,5,237,2021-03-16 14:34:32,2021-03-16 14:42:17,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,160.0,...,19.472648,16.262511,15.610789,13.272727,14.272727,123.769240,125.306434,6377.0,162.972735,Comprehensive - Forced Air + WH + Insulation +...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,136,2021-03-16 13:27:28,2021-03-16 13:28:14,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
2460,2461,202,2021-03-16 14:32:51,2021-03-16 14:33:34,Invalid,True,Comprehensive - Forced Air + WH + Insulation +...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Insulation +...
2461,2462,127,2021-03-16 13:15:09,2021-03-16 13:22:59,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,100.0,...,5.441984,6.660913,5.444993,15.000000,14.545455,62.036305,47.153808,4456.0,77.062611,Comprehensive - Forced Air + WH + Insulation +...
2462,2463,189,2021-03-16 13:46:43,2021-03-16 13:54:24,Success,True,Comprehensive - Forced Air + WH + Insulation +...,,True,160.0,...,15.044415,12.430584,11.333115,12.545455,11.454545,109.745586,206.997408,5343.0,237.919506,Comprehensive - Forced Air + WH + Insulation +...


In [23]:
pkg_no = 13
upgrade_list = [15, 16]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package13.csv, from upgrades: [15, 16]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,230,2021-03-16 14:10:27,2021-03-16 14:11:08,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
1,2,77,2021-03-16 13:21:59,2021-03-16 13:22:41,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
2,3,171,2021-03-16 13:26:43,2021-03-16 13:27:25,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
3,4,213,2021-03-16 13:33:39,2021-03-16 13:34:23,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 2
4,5,125,2021-03-16 13:43:52,2021-03-16 13:44:35,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,117,2021-03-16 13:07:08,2021-03-16 13:11:57,Success,True,Comprehensive - Hydronic + WH + Shell,,True,20.0,...,2.388173,2.396770,2.368519,16.727273,17.000000,5.715720,7.552454,8561.0,8.414891,Comprehensive - Hydronic + WH + Shell - Part 1
2460,2461,61,2021-03-16 13:53:03,2021-03-16 14:00:40,Success,True,Comprehensive - Hydronic + WH + Shell,,True,240.0,...,28.231271,19.297523,19.762855,15.818182,18.818182,110.336867,82.498279,3955.0,124.832757,Comprehensive - Hydronic + WH + Shell - Part 1
2461,2462,213,2021-03-16 14:22:39,2021-03-16 14:23:24,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
2462,2463,218,2021-03-16 13:49:23,2021-03-16 13:50:07,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1


In [24]:
pkg_no = 14
upgrade_list = [17, 18, 19, 20]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50'],
    ['20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
    ['7 ACH50','6 ACH50','5 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)



* package14.csv, from upgrades: [17, 18, 19, 20]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 694, part 2: 1440, part 3: 265, part 4: 65, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,125,2021-03-16 13:47:13,2021-03-16 13:47:56,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
1,2,154,2021-03-16 12:51:24,2021-03-16 12:52:06,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
2,3,138,2021-03-16 13:55:31,2021-03-16 13:56:17,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
3,4,188,2021-03-16 13:55:54,2021-03-16 13:56:36,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
4,5,62,2021-03-16 13:06:06,2021-03-16 13:06:47,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,144,2021-03-16 13:44:10,2021-03-16 13:49:35,Success,True,Comprehensive - Hydronic + WH + Shell + Applia...,,True,20.0,...,2.388173,2.398464,2.364251,14.000000,17.090909,13.668313,15.719434,547.0,16.739358,Comprehensive - Hydronic + WH + Shell + Applia...
2460,2461,54,2021-03-16 13:56:59,2021-03-16 14:04:36,Success,True,Comprehensive - Hydronic + WH + Shell + Applia...,,True,240.0,...,27.870630,19.319988,19.665668,11.727273,17.181818,173.146865,155.910612,4527.0,208.323777,Comprehensive - Hydronic + WH + Shell + Applia...
2461,2462,72,2021-03-16 13:32:19,2021-03-16 13:33:02,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...
2462,2463,117,2021-03-16 12:56:55,2021-03-16 12:57:37,Invalid,True,Comprehensive - Hydronic + WH + Shell + Applia...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell + Applia...


In [25]:
pkg_no = 15
upgrade_list = [21, 22]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package15.csv, from upgrades: [21, 22]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,207,2021-03-16 14:36:23,2021-03-16 14:43:59,Success,True,Comprehensive - ASHP,,True,600.0,...,32.839928,79.563631,29.751048,16.909091,8.272727,273.595674,720.593437,628.0,817.998265,Comprehensive - ASHP - Part 1
1,2,114,2021-03-16 13:48:53,2021-03-16 13:49:36,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
2,3,188,2021-03-16 14:11:45,2021-03-16 14:19:04,Success,True,Comprehensive - ASHP,,True,400.0,...,27.561613,42.369497,23.520104,16.000000,13.909091,105.128399,495.059491,641.0,589.456221,Comprehensive - ASHP - Part 1
3,4,222,2021-03-16 13:16:20,2021-03-16 13:17:00,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 2
4,5,182,2021-03-16 14:21:14,2021-03-16 14:28:42,Success,True,Comprehensive - ASHP,,True,160.0,...,17.434294,25.773450,15.463247,13.272727,14.818182,250.149390,279.779271,6377.0,356.248960,Comprehensive - ASHP - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,127,2021-03-16 14:02:19,2021-03-16 14:03:06,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
2460,2461,203,2021-03-16 14:07:53,2021-03-16 14:08:36,Invalid,True,Comprehensive - ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - ASHP - Part 1
2461,2462,7,2021-03-16 12:11:08,2021-03-16 12:19:08,Success,True,Comprehensive - ASHP,,True,100.0,...,5.447520,16.270329,5.529713,14.272727,9.727273,41.855372,252.273827,618.0,275.904193,Comprehensive - ASHP - Part 1
2462,2463,129,2021-03-16 13:01:26,2021-03-16 13:09:20,Success,True,Comprehensive - ASHP,,True,160.0,...,13.485809,30.042905,11.286811,13.636364,11.545455,70.163292,456.503681,619.0,516.440691,Comprehensive - ASHP - Part 1


In [26]:
pkg_no = 16
upgrade_list = [23, 24]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None #4

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package16.csv, from upgrades: [23, 24]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,204,2021-03-16 13:17:51,2021-03-16 13:18:31,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
1,2,214,2021-03-16 13:59:04,2021-03-16 14:05:30,Success,True,Comprehensive - MSHP,,True,200.0,...,22.085128,19.894985,20.623162,16.090909,8.000000,72.591222,207.004129,632.0,258.149646,Comprehensive - MSHP - Part 1
2,3,210,2021-03-16 14:13:41,2021-03-16 14:14:22,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
3,4,36,2021-03-16 12:58:58,2021-03-16 12:59:41,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 2
4,5,58,2021-03-16 12:56:17,2021-03-16 12:57:01,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,218,2021-03-16 13:43:01,2021-03-16 13:47:48,Success,True,Comprehensive - MSHP,,True,20.0,...,2.869047,5.304882,2.448138,16.363636,9.636364,9.549627,30.070973,631.0,35.297364,Comprehensive - MSHP - Part 1
2460,2461,44,2021-03-16 13:40:21,2021-03-16 13:48:02,Success,True,Comprehensive - MSHP,,True,240.0,...,21.601862,36.191115,19.315260,15.000000,13.545455,86.560147,308.812307,619.0,377.433426,Comprehensive - MSHP - Part 1
2461,2462,160,2021-03-16 13:15:09,2021-03-16 13:15:54,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1
2462,2463,150,2021-03-16 12:54:48,2021-03-16 12:55:31,Invalid,True,Comprehensive - MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - MSHP - Part 1


#### (3) Packages WITH multi-level partitions (N=4, pkg 17-20)

In [27]:
def combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option=None, 
                                         fix_upgrade_name=False, nullify_total_ng=False, save_as_csv=True, 
                                         recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        level1_list (list of dict): list of dictionaries defining the level 1 partition key and options
        level2_list (list of dict): list of dictionaries defining the level 2 partition key and options
        MSHP_option (int): option number in package to correct, default to None
        fix_upgrade_name (bool): default to True, for pkg 21-24 only
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N_upgrades = len(upgrade_list)
    N_partitions = len(level1_list)*len(level2_list)
    if N_upgrades != N_partitions:
        raise ValueError(f'The size of "upgrade_list" = {N_upgrades} does not match the number of enumeration from '+
                        f'"level1_list" and "level2_list" = {N_partitions}')
    
    if not os.path.exists(filename) or recreate_from_scratch:
        
        print(f'building from resstock run results, {N_partitions} partitions...\n')

        ## building ids for each partitions
        bldg_list = []; CBi_len = 0

        ##### level 1
        P4C1 = level1_list # <----
        for i , Ci in enumerate(P4C1,1):
            CBi = set()
            for key, lst in Ci.items():
                Bi = res[res[f'build_existing_model.{key}'].replace({np.nan:'None'}).isin(lst)]['building_id']
                CBi = CBi.union(set(list(Bi)))
                print(f'- 1.{i} {key}, {len(Bi)} / {len(res)}')
            print(f'- 1.{i} total, {len(CBi)}')
            res1 = res[res[f'building_id'].isin(CBi)]

            ##### level 2
            P4C2 = level2_list # <----
            CBi_2all = set(); CBi_2len = 0
            for k, Ci in enumerate(P4C2,1):
                if k == len(P4C2):
                    CBi = set(list(res1['building_id'])).difference(CBi_2all)
                    print(f'    + 3.{k} total, {len(CBi)}')
                else:
                    CBi = set()
                    for key, lst in Ci.items():
                        Bi = res1[res1[f'build_existing_model.{key}'].isin(lst)]['building_id']
                        CBi = CBi.union(set(list(Bi)))
                        print(f'    + 3.{k} {key}, {len(Bi)} / {len(res1)}')
                CBi_2all = CBi_2all.union(CBi)
                bldg_list.append(CBi)

                CBi_2len += len(CBi)
                CBi_len += len(CBi)
                print(f'appending total {len(CBi)}, (1.{i})(2.{k})_culm = {CBi_2len}, overall_culm = {CBi_len}\n')

        ## (2) combine partitions, update upgrade name
        P4 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P4.append(p[p['building_id'].isin(Bi)])

        P4 = pd.concat(P4, axis=0).sort_index()

        P4['apply_upgrade.upgrade_part'] = P4['apply_upgrade.upgrade_name'] # new col to show partition #
        P4['apply_upgrade.upgrade_name'] = P4['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P4.columns:
                P4['simulation_output_report.upgrade_cost_usd_orig'] = P4['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P4['simulation_output_report.upgrade_cost_usd'] = P4['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P4['simulation_output_report.upgrade_cost_usd'] -= P4[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P4['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P4.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P4.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P4['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P4.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P4['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, MSHP_cost]

            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P4.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) fix upgrade name
        try:
            P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']] = \
                P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']].apply(
                lambda x: x.str.replace('low-E Window','Low-Gain Window')) # fix upgrade name
        except:
            pass
        
        ## (5) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P4.loc[P4['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (6) save
        P4.to_csv(filename, index=False)

    else:              
        P4 = pd.read_csv(filename)
    
    return P4

print('func "combine_df_from_two_level_partitions" loaded')

func "combine_df_from_two_level_partitions" loaded


In [28]:
pkg_no = 17 # <----
upgrade_list = list(range(25, 25+8)) # <----
MSHP_option = None 
fix_upgrade_name = False
    
## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package17.csv, from upgrades: [25, 26, 27, 28, 29, 30, 31, 32]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 204 / 694
appending total 204, (1.1)(2.1)_culm = 204, overall_culm = 204

    + 3.2 total, 490
appending total 490, (1.1)(2.2)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 215 / 1440
appending total 215, (1.2)(2.1)_culm = 215, overall_culm = 909

    + 3.2 total, 1225
appending total 1225, (1.2)(2.2)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 19 / 265
appending total 19, (1.3)(2.1)_culm = 19, overall_culm = 2153

    + 3.2 total, 246
appending total 246, (1.3)(2.2)_culm = 265, overall_culm = 2399

- 1.4 infiltration, 65 / 2464
- 1.4 total, 65
    + 3.1 insulation_unfinished_attic, 2 / 65
appending total 2, (1.4)(2.1)_culm = 2, overal

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,182,2021-03-16 13:58:57,2021-03-16 14:09:30,Success,True,Electrification - Gas to ASHP,,True,600.0,...,33.222186,80.056281,29.620467,11.727273,13.272727,413.011920,799.319895,894.0,1026.456715,Electrification - Gas to ASHP - Part 4
1,2,103,2021-03-16 13:59:08,2021-03-16 13:59:51,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 4
2,3,173,2021-03-16 13:16:43,2021-03-16 13:26:14,Success,True,Electrification - Gas to ASHP,,True,400.0,...,27.331106,46.284073,23.764770,16.181818,15.181818,180.789277,579.541085,175.0,860.563131,Electrification - Gas to ASHP - Part 4
3,4,101,2021-03-16 13:19:35,2021-03-16 13:20:15,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 6
4,5,237,2021-03-16 13:28:07,2021-03-16 13:39:18,Success,True,Electrification - Gas to ASHP,,True,160.0,...,17.652142,26.493104,15.760442,12.909091,13.272727,176.231217,239.785238,8478.0,321.096599,Electrification - Gas to ASHP - Part 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,37,2021-03-16 12:26:51,2021-03-16 12:27:33,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 2
2460,2461,120,2021-03-16 13:58:53,2021-03-16 13:59:38,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 4
2461,2462,126,2021-03-16 13:18:15,2021-03-16 13:29:00,Success,True,Electrification - Gas to ASHP,,True,100.0,...,5.506188,17.809476,5.718277,13.818182,12.636364,73.105818,252.907543,634.0,302.857288,Electrification - Gas to ASHP - Part 1
2462,2463,165,2021-03-16 13:35:44,2021-03-16 13:46:01,Success,True,Electrification - Gas to ASHP,,True,160.0,...,13.684424,30.680933,11.420677,10.636364,11.454545,172.028935,450.035205,871.0,585.227120,Electrification - Gas to ASHP - Part 1


In [29]:
pkg_no = 18 # <----
upgrade_list = list(range(33, 33+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package18.csv, from upgrades: [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 128 / 694
appending total 128, (1.1)(2.1)_culm = 128, overall_culm = 128

    + 3.2 insulation_unfinished_attic, 228 / 694
appending total 228, (1.1)(2.2)_culm = 356, overall_culm = 356

    + 3.3 total, 338
appending total 338, (1.1)(2.3)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 85 / 1440
appending total 85, (1.2)(2.1)_culm = 85, overall_culm = 779

    + 3.2 insulation_unfinished_attic, 454 / 1440
appending total 454, (1.2)(2.2)_culm = 539, overall_culm = 1233

    + 3.3 total, 901
appending total 901, (1.2)(2.3)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 8 / 265
appending total 8, (1.3)(2.1)_culm = 8, overall_

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,237,2021-03-16 14:38:49,2021-03-16 14:39:30,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
1,2,69,2021-03-16 13:52:01,2021-03-16 14:02:02,Success,True,Electrification - Gas to MSHP,,True,200.0,...,22.450264,21.393291,20.457838,13.000000,11.909091,239.684893,325.215354,6182.0,363.271572,Electrification - Gas to MSHP - Part 5
2,3,193,2021-03-16 13:39:47,2021-03-16 13:40:30,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
3,4,25,2021-03-16 13:44:46,2021-03-16 13:45:26,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 9
4,5,68,2021-03-16 13:03:09,2021-03-16 13:03:53,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,153,2021-03-16 12:51:14,2021-03-16 12:57:49,Success,True,Electrification - Gas to MSHP,,True,20.0,...,2.924333,5.443059,2.508419,15.727273,13.636364,24.642730,42.622046,631.0,50.595685,Electrification - Gas to MSHP - Part 3
2460,2461,126,2021-03-16 13:14:58,2021-03-16 13:25:40,Success,True,Electrification - Gas to MSHP,,True,240.0,...,21.697097,39.284186,19.579095,12.090909,16.090909,160.928813,399.929687,886.0,497.402644,Electrification - Gas to MSHP - Part 5
2461,2462,118,2021-03-16 14:05:28,2021-03-16 14:06:12,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 2
2462,2463,61,2021-03-16 13:25:16,2021-03-16 13:25:57,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 2


In [30]:
pkg_no = 19 # <----
upgrade_list = list(range(45, 45+8)) # <----
MSHP_option = None
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package19.csv, from upgrades: [45, 46, 47, 48, 49, 50, 51, 52]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 204 / 694
appending total 204, (1.1)(2.1)_culm = 204, overall_culm = 204

    + 3.2 total, 490
appending total 490, (1.1)(2.2)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 215 / 1440
appending total 215, (1.2)(2.1)_culm = 215, overall_culm = 909

    + 3.2 total, 1225
appending total 1225, (1.2)(2.2)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 19 / 265
appending total 19, (1.3)(2.1)_culm = 19, overall_culm = 2153

    + 3.2 total, 246
appending total 246, (1.3)(2.2)_culm = 265, overall_culm = 2399

- 1.4 infiltration, 65 / 2464
- 1.4 total, 65
    + 3.1 insulation_unfinished_attic, 2 / 65
appending total 2, (1.4)(2.1)_culm = 2, overal

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,88,2021-03-16 13:35:10,2021-03-16 13:35:54,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
1,2,243,2021-03-16 13:54:57,2021-03-16 13:55:43,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
2,3,165,2021-03-16 14:12:18,2021-03-16 14:13:03,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
3,4,72,2021-03-16 13:12:11,2021-03-16 13:21:58,Success,True,Electrification - Electric to ASHP,,True,120.0,...,7.038717,17.711006,6.54386,14.454545,17.818182,107.82821,162.41569,544.0,229.013008,Electrification - Electric to ASHP - Part 6
4,5,226,2021-03-16 13:18:49,2021-03-16 13:19:31,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,167,2021-03-16 13:15:01,2021-03-16 13:15:48,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 2
2460,2461,108,2021-03-16 13:27:34,2021-03-16 13:28:24,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 4
2461,2462,95,2021-03-16 13:29:23,2021-03-16 13:30:10,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 1
2462,2463,218,2021-03-16 13:28:17,2021-03-16 13:29:02,Invalid,True,Electrification - Electric to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP - Part 1


In [31]:
pkg_no = 20 # <----
upgrade_list = list(range(53, 53+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)



* package20.csv, from upgrades: [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 128 / 694
appending total 128, (1.1)(2.1)_culm = 128, overall_culm = 128

    + 3.2 insulation_unfinished_attic, 228 / 694
appending total 228, (1.1)(2.2)_culm = 356, overall_culm = 356

    + 3.3 total, 338
appending total 338, (1.1)(2.3)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 85 / 1440
appending total 85, (1.2)(2.1)_culm = 85, overall_culm = 779

    + 3.2 insulation_unfinished_attic, 454 / 1440
appending total 454, (1.2)(2.2)_culm = 539, overall_culm = 1233

    + 3.3 total, 901
appending total 901, (1.2)(2.3)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 8 / 265
appending total 8, (1.3)(2.1)_culm = 8, overall_

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,11,2021-03-16 13:14:41,2021-03-16 13:15:22,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
1,2,228,2021-03-16 13:18:49,2021-03-16 13:19:33,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 5
2,3,178,2021-03-16 13:06:45,2021-03-16 13:07:27,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 6
3,4,9,2021-03-16 12:51:06,2021-03-16 12:51:48,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 9
4,5,33,2021-03-16 12:26:34,2021-03-16 12:27:21,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,20,2021-03-16 13:16:34,2021-03-16 13:17:21,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 3
2460,2461,137,2021-03-16 14:04:57,2021-03-16 14:05:45,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 5
2461,2462,50,2021-03-16 12:33:15,2021-03-16 12:34:01,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 2
2462,2463,16,2021-03-16 12:12:58,2021-03-16 12:13:39,Invalid,True,Electrification - Electric to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP - Part 2


In [32]:
# with low-gain windows
pkg_no = 21 # <----
upgrade_list = list(range(65, 65+8)) # <----
MSHP_option = None 
fix_upgrade_name = True
    
## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)



* package21.csv, from upgrades: [65, 66, 67, 68, 69, 70, 71, 72]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 204 / 694
appending total 204, (1.1)(2.1)_culm = 204, overall_culm = 204

    + 3.2 total, 490
appending total 490, (1.1)(2.2)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 215 / 1440
appending total 215, (1.2)(2.1)_culm = 215, overall_culm = 909

    + 3.2 total, 1225
appending total 1225, (1.2)(2.2)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 19 / 265
appending total 19, (1.3)(2.1)_culm = 19, overall_culm = 2153

    + 3.2 total, 246
appending total 246, (1.3)(2.2)_culm = 265, overall_culm = 2399

- 1.4 infiltration, 65 / 2464
- 1.4 total, 65
    + 3.1 insulation_unfinished_attic, 2 / 65
appending total 2, (1.4)(2.1)_culm = 2, overal

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,74,2021-03-16 12:42:39,2021-03-16 12:53:48,Success,True,Electrification - Gas to ASHP (Low-Gain Window),,True,600.0,...,33.072273,84.904513,29.548021,12.454545,12.181818,411.471806,827.379727,895.0,1054.589599,Electrification - Gas to ASHP (Low-Gain Window...
1,2,126,2021-03-16 13:17:22,2021-03-16 13:18:05,Invalid,True,Electrification - Gas to ASHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP (Low-Gain Window...
2,3,158,2021-03-16 13:18:15,2021-03-16 13:27:48,Success,True,Electrification - Gas to ASHP (Low-Gain Window),,True,400.0,...,27.231154,46.776477,23.754271,15.272727,15.181818,184.619696,586.126909,175.0,859.463536,Electrification - Gas to ASHP (Low-Gain Window...
3,4,127,2021-03-16 12:59:59,2021-03-16 13:00:39,Invalid,True,Electrification - Gas to ASHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP (Low-Gain Window...
4,5,219,2021-03-16 14:04:25,2021-03-16 14:15:23,Success,True,Electrification - Gas to ASHP (Low-Gain Window),,True,160.0,...,17.581558,27.258003,15.725805,14.000000,12.272727,177.429750,240.566929,8478.0,320.606035,Electrification - Gas to ASHP (Low-Gain Window...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,88,2021-03-16 12:47:19,2021-03-16 12:48:00,Invalid,True,Electrification - Gas to ASHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP (Low-Gain Window...
2460,2461,138,2021-03-16 12:50:12,2021-03-16 12:50:59,Invalid,True,Electrification - Gas to ASHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP (Low-Gain Window...
2461,2462,52,2021-03-16 13:02:26,2021-03-16 13:12:51,Success,True,Electrification - Gas to ASHP (Low-Gain Window),,True,100.0,...,5.468106,18.231517,5.718876,13.818182,10.181818,73.782118,252.060552,634.0,307.245783,Electrification - Gas to ASHP (Low-Gain Window...
2462,2463,158,2021-03-16 13:15:48,2021-03-16 13:25:43,Success,True,Electrification - Gas to ASHP (Low-Gain Window),,True,160.0,...,13.602932,31.160182,11.430784,10.636364,11.545455,171.325895,449.527934,871.0,583.434519,Electrification - Gas to ASHP (Low-Gain Window...


In [33]:
# with low-gain windows
pkg_no = 22 # <----
upgrade_list = list(range(73, 73+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = True

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package22.csv, from upgrades: [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 128 / 694
appending total 128, (1.1)(2.1)_culm = 128, overall_culm = 128

    + 3.2 insulation_unfinished_attic, 228 / 694
appending total 228, (1.1)(2.2)_culm = 356, overall_culm = 356

    + 3.3 total, 338
appending total 338, (1.1)(2.3)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 85 / 1440
appending total 85, (1.2)(2.1)_culm = 85, overall_culm = 779

    + 3.2 insulation_unfinished_attic, 454 / 1440
appending total 454, (1.2)(2.2)_culm = 539, overall_culm = 1233

    + 3.3 total, 901
appending total 901, (1.2)(2.3)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 8 / 265
appending total 8, (1.3)(2.1)_culm = 8, overall_

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,92,2021-03-16 13:52:15,2021-03-16 13:52:55,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...
1,2,126,2021-03-16 12:56:43,2021-03-16 13:06:10,Success,True,Electrification - Gas to MSHP (Low-Gain Window),,True,200.0,...,22.296950,21.382469,20.478548,13.000000,11.454545,235.572115,337.714722,636.0,367.076174,Electrification - Gas to MSHP (Low-Gain Window...
2,3,224,2021-03-16 13:50:17,2021-03-16 13:51:04,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...
3,4,133,2021-03-16 14:00:46,2021-03-16 14:01:28,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...
4,5,128,2021-03-16 13:46:20,2021-03-16 13:47:06,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,219,2021-03-16 13:56:52,2021-03-16 14:03:13,Success,True,Electrification - Gas to MSHP (Low-Gain Window),,True,20.0,...,2.911501,5.562895,2.513390,16.090909,14.363636,24.505709,43.608950,631.0,51.688716,Electrification - Gas to MSHP (Low-Gain Window...
2460,2461,185,2021-03-16 13:47:23,2021-03-16 13:58:32,Success,True,Electrification - Gas to MSHP (Low-Gain Window),,True,240.0,...,21.625916,40.296785,19.604480,12.090909,16.272727,160.625535,407.574534,886.0,503.884613,Electrification - Gas to MSHP (Low-Gain Window...
2461,2462,48,2021-03-16 13:05:54,2021-03-16 13:06:40,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...
2462,2463,130,2021-03-16 13:45:46,2021-03-16 13:46:32,Invalid,True,Electrification - Gas to MSHP (Low-Gain Window),,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP (Low-Gain Window...


In [34]:
# with low-gain windows
pkg_no = 23 # <----
upgrade_list = list(range(85, 85+8)) # <----
MSHP_option = None
fix_upgrade_name = True

## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package23.csv, from upgrades: [85, 86, 87, 88, 89, 90, 91, 92]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 204 / 694
appending total 204, (1.1)(2.1)_culm = 204, overall_culm = 204

    + 3.2 total, 490
appending total 490, (1.1)(2.2)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 215 / 1440
appending total 215, (1.2)(2.1)_culm = 215, overall_culm = 909

    + 3.2 total, 1225
appending total 1225, (1.2)(2.2)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 19 / 265
appending total 19, (1.3)(2.1)_culm = 19, overall_culm = 2153

    + 3.2 total, 246
appending total 246, (1.3)(2.2)_culm = 265, overall_culm = 2399

- 1.4 infiltration, 65 / 2464
- 1.4 total, 65
    + 3.1 insulation_unfinished_attic, 2 / 65
appending total 2, (1.4)(2.1)_culm = 2, overal

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,74,2021-03-16 13:57:15,2021-03-16 13:57:57,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
1,2,26,2021-03-16 12:59:03,2021-03-16 12:59:47,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
2,3,100,2021-03-16 12:58:15,2021-03-16 12:58:58,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
3,4,98,2021-03-16 13:11:26,2021-03-16 13:22:05,Success,True,Electrification - Electric to ASHP (Low-Gain W...,,True,120.0,...,6.86308,19.243654,6.522813,14.454545,18.727273,106.143812,165.562601,544.0,228.177909,Electrification - Electric to ASHP (Low-Gain W...
4,5,104,2021-03-16 13:23:30,2021-03-16 13:24:14,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,143,2021-03-16 13:46:11,2021-03-16 13:47:00,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
2460,2461,133,2021-03-16 14:02:08,2021-03-16 14:02:53,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
2461,2462,187,2021-03-16 14:00:04,2021-03-16 14:00:48,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...
2462,2463,170,2021-03-16 14:05:11,2021-03-16 14:05:51,Invalid,True,Electrification - Electric to ASHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to ASHP (Low-Gain W...


In [35]:
# with low-gain windows
pkg_no = 24 # <----
upgrade_list = list(range(93, 93+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = True

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package24.csv, from upgrades: [93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 128 / 694
appending total 128, (1.1)(2.1)_culm = 128, overall_culm = 128

    + 3.2 insulation_unfinished_attic, 228 / 694
appending total 228, (1.1)(2.2)_culm = 356, overall_culm = 356

    + 3.3 total, 338
appending total 338, (1.1)(2.3)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 85 / 1440
appending total 85, (1.2)(2.1)_culm = 85, overall_culm = 779

    + 3.2 insulation_unfinished_attic, 454 / 1440
appending total 454, (1.2)(2.2)_culm = 539, overall_culm = 1233

    + 3.3 total, 901
appending total 901, (1.2)(2.3)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 8 / 265
appending total 8, (1.3)(2.1)_culm = 8, ove

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,185,2021-03-16 13:36:02,2021-03-16 13:36:45,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
1,2,100,2021-03-16 13:19:09,2021-03-16 13:19:51,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
2,3,72,2021-03-16 13:23:40,2021-03-16 13:24:24,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
3,4,72,2021-03-16 13:08:52,2021-03-16 13:09:36,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
4,5,6,2021-03-16 13:15:30,2021-03-16 13:16:15,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,108,2021-03-16 13:39:29,2021-03-16 13:40:11,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
2460,2461,220,2021-03-16 14:45:10,2021-03-16 14:45:55,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
2461,2462,106,2021-03-16 12:48:07,2021-03-16 12:48:50,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...
2462,2463,2,2021-03-16 12:34:46,2021-03-16 12:35:28,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Electric to MSHP (Low-Gain W...


## 2B. PACKAGE summary ###

In [36]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

funcs loaded


In [37]:
### set utility rates ###
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''

### utility rates ###########################################
# ref (EIA): 
# avg ComEd res elec rate 2019: $ 0.1330 /kWh
# weighted avg IL gas rate 2019: $ 0.77183 /therm

## electricity ##
# annual fixed rates = monthly x 12
EH_fixed = 15.70 * 12 # annual
NEH_fixed = 14.28 *12 # annual
# avg of summer rates (J,J,A,S) and non-summer rates
EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# marginal carbon emission factor:
elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

## gas ##
# annual fixed rates = monthly x 12
GH_fixed = 45.32 * 12
NGH_fixed = 21.51 * 12
# variable rates
GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# marginal carbon emission factor:
gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

### upgrades that will cause utility rate change: ###########################################
HVAC_upgrades_rate_change = {
    15: [4,5,6], # ASHP
    16: [4,5,6,7], # MSHP
    17: [5,6,7], # ASHP
    18: [5,6,7,8], # MSHP,
    21: [5,6,7], # ASHP
    22: [5,6,7,8], # MSHP,
}

# assign rates accordingly
Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
                      'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
                    'Natural Gas Fuel Boiler']

res['gas_rate'] = NGH_rate
res['gas_fixed'] = NGH_fixed
res['gas_CO2_rate'] = gas_CO2_rate
res['elec_rate'] = NEH_rate
res['elec_fixed'] = NEH_fixed
res['elec_CO2_rate'] = elec_CO2_rate

res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_rate'] = GH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_fixed'] = GH_fixed
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_rate'] = EH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_fixed'] = EH_fixed

# assign 0 gas rates to building with no gas use 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
    res.loc[res['completed_status']!='Success', col] = np.nan

print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


Natural gas rate multiplier: 1


In [38]:
#### count upgrades
N_upgrades = 19 # <---
export_all_upgrades_to_csv = True # <------

print(f'>>> {iter_path} has {N_upgrades:,} packages')
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
if export_all_upgrades_to_csv:
    print(f'    Exporting upgrades results to {combined_res_csv_path}\n')        
    res.to_csv(os.path.join(combined_res_csv_path, f'results_baseline{fn_ext}.csv'), index=False)
    
## get summary table
summary_upgrades = []

## initialize dB table
all_proto_upgrades = []

package_list = list(range(6, N_upgrades+6))
for n in package_list:
    p = pd.read_csv(os.path.join(combined_res_csv_path,
                                     f'package{n:02d}.csv'))
    print(f'\nPackage {n}')
    p['build_existing_model.sample_weight'] = 2173432/40000
    
    ### get sim output at unit level
    p = get_per_unit_sim_output_limited(p, res)

    ### assign utility rates
    p = assign_utility_rates_to_upgrade(n, p, res, HVAC_upgrades_rate_change, for_packages=True)
    
    ### collapse upgrade cost and lifetime cols
    p = combine_upgrade_cost_and_lifetime(p)

    ### check if upgrade has 0 successful sims
    if len(p[p['completed_status']=='Success']) == 0:
        print(f' * upgrade={n} has 0 successful simulations')

    ### calculate metrics
    p = add_sqft_eui(p, res)
    EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']
    EUIo = ['gas_eui','elec_eui','site_eui']
    for vari, varo in zip(EUIi, EUIo):
        p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)


    # annual energy saving:
    p = get_annual_gas_elec_site_energy_saving(p, res)

    # annual energy cost saving:
    p = get_annual_energy_cost_saving(p, res)
    
    # annual kBtu saved per upgrade cost:
    p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
                            p['upgrade_cost'], axis=0)*1000

    # simple payback
    p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

    # annual metric ton carbon emission savings:
    p = get_annual_metric_ton_co2e_saving(p, res)

    ### check for neg/large paybacks, inf eui, neg carbon savings 
    print_metrics_report(p, has_comparative_payback=False)
    
    ### subset to only those that have been applied with the upgrades successfully:
    upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
    p = p[p['apply_upgrade.applicable']==True].reset_index(drop=True)
    p['package_no'] = n
    
    ### export upgrade results to csv 
    if export_all_upgrades_to_csv:
        p.to_csv(os.path.join(combined_res_csv_path, f'results_package{n:02d}{fn_ext}.csv'), index=False)
    
    ### add to dB
    all_proto_upgrades.append(p)
    
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc
        
    ### add to summary table
    summ = pd.DataFrame()
    summ.loc[0,'upgrade_no'] = n
    summ.loc[0,'upgrade_name'] = upgrade_name
    summ.loc[0,'n_applied'] = len(p)
    summ.loc[0,'n_success'] = len(p[p['completed_status']=='Success'])
    summ.loc[0,'n_fail'] = len(p[p['completed_status']=='Fail'])
    summ['pct_success'] = round(summ['n_success']/summ['n_applied']*100,3)

    p = p[p['completed_status']=='Success'].reset_index(drop=True)
    summ['mean_ann_therm_gas_saving'] = round(p['ann_therm_gas_saving'].mean(), 3)
    summ['mean_ann_kwh_elec_saving'] = round(p['ann_kwh_elec_saving'].mean(), 3)
    summ['mean_ann_mbtu_site_energy_saving'] = round(p['ann_mbtu_site_energy_saving'].mean(), 3)
    summ['mean_pct_delta_gas_eui'] = round(p['pct_delta_gas_eui'].mean(), 3)
    summ['mean_pct_delta_elec_eui'] = round(p['pct_delta_elec_eui'].mean(), 3)
    summ['mean_pct_delta_site_eui'] = round(p['pct_delta_site_eui'].mean(), 3)
    summ['mean_upgrade_cost'] = round(p['upgrade_cost'].mean(),2)
    summ['mean_ann_kbtu_saved_per_dollar'] = round(p['ann_kbtu_saved_per_dollar'].mean(),3) # annual kBtu saved per upgrade cost
    summ['mean_ann_energy_cost_saving'] = round(p['ann_energy_cost_saving'].mean(),2)
    summ['mean_ann_metric_ton_co2e_saving'] = round(p['ann_metric_ton_co2e_saving'].mean(),3)

    summ['median_simple_payback'] = round(p['simple_payback'].median(),3)
    
    p['simple_payback'] = p['simple_payback'].replace([np.inf, -np.inf], np.nan) # for mean calc
    summ['pct_pos_simple_payback_actual'] = round(len(p[p['simple_payback']>=0])/len(p)*100, 3)
    summ['mean_pos_simple_payback_actual'] = round(p.loc[p['simple_payback']>=0, 'simple_payback'].mean(),3)

    # filter: min 1 cent energy cost savings **
    min_energy_cost_saving = 0.1 # <----- **
    summ['pct_pos_simple_payback_filtered'] = round(len(
        p[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving)]
    )/len(p)*100, 3)
    summ['mean_pos_simple_payback_filtered'] = round(
        p.loc[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving), 
           'simple_payback'].mean(),3)

    summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


>>> cookcnty_packages_02_sfamh has 19 packages
Natural gas rate multiplier: 1
    Exporting upgrades results to /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results/raw combined csvs


Package 6
min       -8549.569530
median       19.588915
max       22628.453928
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=6 has 59 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=6 has 103 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=6 has 106 carbon saving<0 

Package 7
min      -17846.170756
median       32.205590
max       22944.517897
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=7 has 128 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=7 has 45 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=7 has 47 carbon saving<0 

Package 8
min         9.528980
median     39.855405


,upgrade_no,upgrade_name,n_applied,n_success,n_fail,pct_success,mean_ann_therm_gas_saving,mean_ann_kwh_elec_saving,mean_ann_mbtu_site_energy_saving,mean_pct_delta_gas_eui,...,mean_pct_delta_site_eui,mean_upgrade_cost,mean_ann_kbtu_saved_per_dollar,mean_ann_energy_cost_saving,mean_ann_metric_ton_co2e_saving,median_simple_payback,pct_pos_simple_payback_actual,mean_pos_simple_payback_actual,pct_pos_simple_payback_filtered,mean_pos_simple_payback_filtered
0,6.0,Utility Incentive Shell + LEDs,2464.0,2464.0,0.0,100.000,112.518,934.162,14.544,-12.481,...,-12.851,3530.62,3.813,181.93,1.379,19.589,95.820,50.542,95.820,50.542
1,7.0,Utility Incentive Shell + LEDs - Forced Air,2464.0,1947.0,0.0,79.018,158.446,705.389,18.248,-18.248,...,-16.673,5844.99,2.978,186.63,1.432,32.206,97.689,79.260,97.689,79.260
2,8.0,Utility Incentive Shell + LEDs + Appliances - ...,2464.0,1947.0,0.0,79.018,263.988,1024.405,29.888,-33.786,...,-29.151,10796.82,2.664,291.90,2.260,39.855,100.000,48.065,100.000,48.065
3,9.0,Utility Incentive Shell + WH - Hydronic,2464.0,230.0,0.0,9.334,363.500,669.160,38.625,-38.042,...,-32.219,11652.49,2.938,308.54,2.493,44.229,100.000,50.722,100.000,50.722
4,10.0,Utility Incentive Shell + LEDs + Appliances - ...,2464.0,230.0,0.0,9.334,374.170,907.724,40.505,-39.615,...,-34.328,14394.41,2.522,345.23,2.749,47.938,100.000,52.921,100.000,52.921
5,11.0,Comprehensive - Forced Air + WH + Shell,2464.0,1947.0,0.0,79.018,285.982,823.582,31.402,-36.562,...,-30.393,12327.53,2.477,280.10,2.209,47.226,99.949,62.329,99.949,62.329
6,12.0,Comprehensive - Forced Air + WH + Insulation +...,2464.0,1947.0,0.0,79.018,284.227,735.022,30.924,-37.008,...,-30.272,11279.39,2.652,267.84,2.126,44.983,99.538,68.420,99.538,68.420
7,13.0,Comprehensive - Hydronic + WH + Shell,2464.0,230.0,0.0,9.334,395.296,680.031,41.841,-41.815,...,-35.159,15752.91,2.403,329.52,2.671,54.637,100.000,61.755,100.000,61.755
8,14.0,Comprehensive - Hydronic + WH + Shell + Applia...,2464.0,230.0,0.0,9.334,395.324,572.273,41.476,-42.638,...,-35.300,14802.79,2.522,315.93,2.582,53.569,100.000,64.722,100.000,64.722
9,15.0,Comprehensive - ASHP,2464.0,1947.0,0.0,79.018,596.471,-3334.280,48.256,-75.824,...,-46.915,11644.78,3.992,410.28,0.391,24.461,96.302,40.232,96.302,40.232


In [39]:
### export UPGRADE summary
summary_upgrades.to_csv(os.path.join(result_path,'processed results',f'upgrades_summary{fn_ext}.csv'), index=False)
print(f'UPGRADE summary table saved to:\n  {os.path.join(result_path, "processed results")}')

UPGRADE summary table saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_02_sfamh/processed results


In [40]:
### optional - export specific upgrade parquet as csv
save_to_csv = False # <-----
upgrade_list = range(1, 28+1) # [8] # <----- list of upgrades to convert to csv

if save_to_csv:
    for n in upgrade_list:
        p = pd.read_parquet(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.parquet'))
        p.to_csv(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.csv'), index=False)
        print(f'upgrade {n:02d} parquet file exported as csv')


### *Combine results by building prototypes from Elevate Energy into database

In [41]:
# (1) combine baseline results by non-split-level prototypes (N=12)
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
### add totals metrics:
res = get_annual_totals(res)

res_proto_upgrades = res.copy()

res_proto_upgrades['package_no'] = 0
res_proto_upgrades['apply_upgrade.upgrade_name'] = 'Baseline'

display(res_proto_upgrades)

Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,elec_rate,elec_fixed,elec_CO2_rate,ann_gas_cost,ann_elec_cost,ann_energy_cost,ann_metric_ton_co2e_gas,ann_metric_ton_co2e_elec,ann_metric_ton_co2e,package_no
0,1,74,2021-03-16 13:37:14,2021-03-16 13:41:12,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,873.858683,793.084218,1666.942901,2.848911,4.111942,6.960853,0
1,2,36,2021-03-16 12:25:01,2021-03-16 12:28:13,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1011.464539,936.954495,1948.419034,4.036804,5.063467,9.100271,0
2,3,42,2021-03-16 13:03:52,2021-03-16 13:07:45,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,875.227182,763.658640,1638.885822,2.860725,3.917328,6.778052,0
3,4,107,2021-03-16 14:01:48,2021-03-16 14:05:09,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,0.000000,1678.047516,1678.047516,0.000000,11.592862,11.592862,0
4,5,79,2021-03-16 14:00:54,2021-03-16 14:04:54,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,841.892848,1255.702138,2097.594985,2.572964,7.171591,9.744554,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,217,2021-03-16 14:23:12,2021-03-16 14:24:31,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1512.427330,1146.905410,2659.332740,8.361403,6.452034,14.813437,0
2460,2461,213,2021-03-16 14:11:20,2021-03-16 14:15:11,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1076.615615,978.101812,2054.717427,4.599226,5.335606,9.934831,0
2461,2462,214,2021-03-16 13:28:31,2021-03-16 13:32:24,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1193.074688,808.424009,2001.498697,5.604567,4.213396,9.817963,0
2462,2463,94,2021-03-16 13:04:33,2021-03-16 13:08:38,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1244.649061,1017.436387,2262.085448,6.049787,5.595756,11.645542,0


In [42]:
# (2) combine upgrade results
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')

all_proto_upgrades = pd.concat(all_proto_upgrades, axis=0).reset_index(drop=True)
all_proto_upgrades


Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no,apply_upgrade.upgrade_part
0,1,167,2021-03-16 14:00:39,2021-03-16 14:07:21,Success,True,Utility Incentive Shell + LEDs,NaN,True,600.0,...,40.813996,110.470791,151.284787,3.011316,21.084995,0.352330,0.730629,1.082958,6,NaN
1,2,17,2021-03-16 12:15:34,2021-03-16 12:20:15,Success,True,Utility Incentive Shell + LEDs,NaN,True,200.0,...,42.593922,75.975865,118.569786,3.449446,21.911147,0.367695,0.502487,0.870182,6,NaN
2,3,23,2021-03-16 13:10:52,2021-03-16 13:15:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,400.0,...,43.795096,24.001336,67.796433,4.030402,28.367274,0.378064,0.158739,0.536804,6,NaN
3,4,122,2021-03-16 13:18:18,2021-03-16 13:23:48,Success,True,Utility Incentive Shell + LEDs,NaN,True,120.0,...,0.000000,-80.275706,-80.275706,-0.879358,-36.177342,0.000000,-0.624728,-0.624728,6,NaN
4,5,107,2021-03-16 13:48:18,2021-03-16 13:54:05,Success,True,Utility Incentive Shell + LEDs,NaN,True,160.0,...,-17.566906,118.343192,100.776286,0.178186,19.528404,-0.151648,0.782695,0.631047,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46811,2460,108,2021-03-16 13:39:29,2021-03-16 13:40:11,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Electrification - Electric to MSHP (Low-Gain W...
46812,2461,220,2021-03-16 14:45:10,2021-03-16 14:45:55,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Electrification - Electric to MSHP (Low-Gain W...
46813,2462,106,2021-03-16 12:48:07,2021-03-16 12:48:50,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Electrification - Electric to MSHP (Low-Gain W...
46814,2463,2,2021-03-16 12:34:46,2021-03-16 12:35:28,Invalid,True,Electrification - Electric to MSHP (Low-Gain W...,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,Electrification - Electric to MSHP (Low-Gain W...


In [43]:
# (3) combine upgrade and baseline

cols = set(res_proto_upgrades.columns).intersection(set(all_proto_upgrades.columns))
all_proto_upgrades = pd.concat([all_proto_upgrades, res_proto_upgrades[cols]], axis=0).reset_index(drop=True)

# (4) save
print('"all_proto_upgrades" df saved')
all_proto_upgrades.to_csv(
    os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv'), index=False)

display(all_proto_upgrades)


"all_proto_upgrades" df saved


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no,apply_upgrade.upgrade_part
0,1,167,2021-03-16 14:00:39,2021-03-16 14:07:21,Success,True,Utility Incentive Shell + LEDs,NaN,True,600.0,...,40.813996,110.470791,151.284787,3.011316,21.084995,0.352330,0.730629,1.082958,6,NaN
1,2,17,2021-03-16 12:15:34,2021-03-16 12:20:15,Success,True,Utility Incentive Shell + LEDs,NaN,True,200.0,...,42.593922,75.975865,118.569786,3.449446,21.911147,0.367695,0.502487,0.870182,6,NaN
2,3,23,2021-03-16 13:10:52,2021-03-16 13:15:51,Success,True,Utility Incentive Shell + LEDs,NaN,True,400.0,...,43.795096,24.001336,67.796433,4.030402,28.367274,0.378064,0.158739,0.536804,6,NaN
3,4,122,2021-03-16 13:18:18,2021-03-16 13:23:48,Success,True,Utility Incentive Shell + LEDs,NaN,True,120.0,...,0.000000,-80.275706,-80.275706,-0.879358,-36.177342,0.000000,-0.624728,-0.624728,6,NaN
4,5,107,2021-03-16 13:48:18,2021-03-16 13:54:05,Success,True,Utility Incentive Shell + LEDs,NaN,True,160.0,...,-17.566906,118.343192,100.776286,0.178186,19.528404,-0.151648,0.782695,0.631047,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49275,2460,217,2021-03-16 14:23:12,2021-03-16 14:24:31,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
49276,2461,213,2021-03-16 14:11:20,2021-03-16 14:15:11,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
49277,2462,214,2021-03-16 13:28:31,2021-03-16 13:32:24,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
49278,2463,94,2021-03-16 13:04:33,2021-03-16 13:08:38,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [44]:
## save metadata for dB
if NG_rate_multiplier==1:
    cols2 = sorted((set(res_proto_upgrades.columns)-set(cols)).union(
                   set(['building_id','completed_status','package_no']))
                  )
    res_proto_meta = res_proto_upgrades[cols2]
    res_proto_meta = res_proto_meta[res_proto_meta['completed_status']=='Success'].reset_index(drop=True)
    
    cols_to_drop = [x for x in res_proto_meta.columns if 
                    x.startswith('simulation_output_report.option') or
                    x == 'simulation_output_report.upgrade_cost_usd']
    
    res_proto_meta = res_proto_meta.drop(cols_to_drop, axis=1)

    res_proto_meta.to_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv'), index=False)

    print('"res_proto_meta" df saved')
    display(res_proto_meta)
    

"res_proto_meta" df saved


,ann_elec_cost,ann_energy_cost,ann_gas_cost,ann_metric_ton_co2e,ann_metric_ton_co2e_elec,ann_metric_ton_co2e_gas,build_existing_model.ahs_region,build_existing_model.applicable,build_existing_model.ashrae_iecc_climate_zone_2004,build_existing_model.bathroom_spot_vent_hour,...,build_existing_model.vintage,build_existing_model.vintage_acs,build_existing_model.water_heater,build_existing_model.window_areas,build_existing_model.windows,building_id,completed_status,package_no,stories,vintage_ee
0,793.084218,1666.942901,873.858683,6.960853,4.111942,2.848911,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour5,...,1990s,1980-99,Gas Standard,F12 B12 L12 R12,2+ Pane,1,Success,0,2: 2+ stories,3: post-1978
1,936.954495,1948.419034,1011.464539,9.100271,5.063467,4.036804,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour3,...,1960s,1960-79,Gas Standard,F12 B12 L12 R12,2+ Pane,2,Success,0,1: <2 stories,2: 1942-1978
2,763.658640,1638.885822,875.227182,6.778052,3.917328,2.860725,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour18,...,2000s,2000-09,Gas Standard,F9 B9 L9 R9,2+ Pane,3,Success,0,2: 2+ stories,3: post-1978
3,1678.047516,1678.047516,0.000000,11.592862,11.592862,0.000000,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour21,...,1990s,1980-99,Electric Standard,F18 B18 L18 R18,2+ Pane,4,Success,0,1: <2 stories,3: post-1978
4,1255.702138,2097.594985,841.892848,9.744554,7.171591,2.572964,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,...,2000s,2000-09,Gas Standard,F9 B9 L9 R9,1 Pane,5,Success,0,2: 2+ stories,3: post-1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,1146.905410,2659.332740,1512.427330,14.813437,6.452034,8.361403,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour21,...,<1940,<1940,Gas Standard,F12 B12 L12 R12,1 Pane,2460,Success,0,1: <2 stories,1: pre-1942
2460,978.101812,2054.717427,1076.615615,9.934831,5.335606,4.599226,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour4,...,1970s,1960-79,Gas Standard,F9 B9 L9 R9,2+ Pane,2461,Success,0,2: 2+ stories,2: 1942-1978
2461,808.424009,2001.498697,1193.074688,9.817963,4.213396,5.604567,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,...,1990s,1980-99,Gas Standard,F6 B6 L6 R6,1 Pane,2462,Success,0,2: 2+ stories,3: post-1978
2462,1017.436387,2262.085448,1244.649061,11.645542,5.595756,6.049787,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour5,...,1990s,1980-99,Gas Standard,F9 B9 L9 R9,1 Pane,2463,Success,0,2: 2+ stories,3: post-1978
